## Raw ideas

so here's a few quick ideas for a validation schema:

- data shouldn't be shuffled before training, cause it's essential to have the time series values in consecutive order
- currently I came up with three ideas for a schema:
    1. `1,2train+3valid` -> `1,2,3train+4valid` -> `1,2,3,4train+5valid` etc.

        **Advatages**:
        - the train set grows, probably helping to test model's robustness
        - we always validate on different parts of the training set

        **Disadvantages**:
        - computationally heavy. at some point our train will contain almost the entire dataset
    2. `1,2train+3valid` -> `2,3train+4valid` -> `3,4train+5valid`

        **Advantages**:
        - fixed train size, no computational power issues
        - we can somewhat efficiently use our dataset since we use each fold several times during different trainig sessions
        
        **Disadvantages**:
        - the fact that different sets overlap might be a problem (shouldn't be, but who knows)
    3.  `1,2train+3valid` -> `4,5train+6valid` -> `7,8train+9valid`

        **Advantages**:
        - the easiest one to implement, goes through each fold once only
        - it's guaranteed that there's no overlapping or data leaks
        
        **Disadvantages**:
        - data hugry in some sense, each fold is used only ones and each trainig session takes $f_{train} + 1$ folds, where $f_{train}$ is the amount of folds for the trainig set. this might be a problem assuming we have only 33 folds at maximum

model ideas:

- **decision tree-based algorithms (boosting, random forest)**:

    seems to be a good choice for the task since we have a lot of categorical features and the target is a discrete variable (which is typical for decision tree regression). at the same time, trees can be overfit easily, which can cause a huge loss in robustness and very inaccurate predictions on unseen data. ensembles will help to resolve this, but the problem of overfitting won't be gone completely. also, decision trees don't work well once something completely unseen shows up in input data, which might be a problem during future usage
- **auto-regressive models**:

    a time-series specific solution, that's supposed to do a good job at predicting future sales based on historical data. the main problems are the following: 
        1. I don't have any experience working with them (but we'll still try I guess)
        2. the data is still quite noisy, so using AR models will require some additional preprocessing 
- **RNNs**:

    one of the hardest models to implement, but at the same time a really powerfull (in theory) solution, that by definition fits the task idea quite well. RNNs can be really good when it comes to analyzing consecitive data (like text processing or time series) 


useful additional features:

- day of the week
- month
- year
- lagged values (look them up in `EDA`)
- ...


In [2]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('../data/processed_files/processed_data.csv')
data['date'] = pd.to_datetime(data['date'])

In [6]:
data['date_block_num'].unique()

33

In [23]:
import pandas as pd

class ModelValidator:
    def __init__(self, data: pd.DataFrame, target_col: str,  n_train_folds: int, n_valid_folds: int = 1, mode: str = 'sequence') -> None:
        """
        TODO add a description

        Keyword arguments:

        data -- a pandas dataframe containing the training data

        target_col -- the name of a column in `data` contains the target values

        n_train_folds -- the amount of months used to train the model.
        must be less or equal to the amount of months in `data` minus n_valid_folds

        n_valid_folds -- the amount of months used to validate the model.
        must be less or equal to the amount of months in `data` minus n_train_folds
        Default: 1

        mode -- string denoting the data splitting mode:
            - `sequence` for `1,2train+3valid` -> `4,5train+6valid` -> `7,8train+9valid`
            - `overlap` for `1,2train+3valid` -> `2,3train+4valid` -> `3,4train+5valid`
            - `stack` for `1,2train+3valid` -> `1,2,3train+4valid` -> `1,2,3,4train+5valid`
        Default: `sequence`
        """
        self.min_fold = data['date_block_num'].unique().min()
        self.max_fold = data['date_block_num'].unique().max()
        self.current_fold = self.min_fold
        self.train_size = n_train_folds
        self.val_size = n_valid_folds
        if (self.max_fold - self.min_fold +1) < self.train_size + self.val_size:
            raise ValueError(f'too many folds in train and valid for the given data')
        self.data = data
        self.mode = mode

    def _extract_features(self, fold) -> pd.DataFrame:
        # questionable, because we might want to use the entire dataset for feature extraction
        # TODO actual feature extraction
        return fold

    def _get_sets(self) -> pd.DataFrame:
        train = pd.DataFrame(columns=self.data.columns)
        val = pd.DataFrame(columns=self.data.columns)
        
        if self.mode.startswith('seq'):
            print('here')
            for idx in range(self.current_fold, self.current_fold + self.train_size):
                print(idx, 'train')
                print(train)

                pd.concat([train, self.data[self.data['date_block_num']==idx]], ignore_index=True)
            self.current_fold += self.train_size
            for idx in range(self.current_fold, self.current_fold + self.val_size):
                pd.concat([val, self.data[self.data['date_block_num'] == idx]], ignore_index=True)
            self.current_fold += self.val_size
            print(train.shape, val.shape)
        
        return (train, val)

    def validate(self, model):
        # conceptually:
        data = self._extract_features(self._get_sets())
        model.fit(data)

        
validator = ModelValidator(data, 'item_cnt_day', 3)
validator._get_sets()

here
0 train
Empty DataFrame
Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id, shop_name, item_category_name]
Index: []
1 train
Empty DataFrame
Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id, shop_name, item_category_name]
Index: []
2 train
Empty DataFrame
Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id, shop_name, item_category_name]
Index: []


/tmp/ipykernel_2335/1475630287.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([train, self.data[self.data['date_block_num']==idx]], ignore_index=True)
/tmp/ipykernel_2335/1475630287.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([val, self.data[self.data['date_block_num'] == idx]], ignore_index=True)


(0, 10) (0, 10)


(Empty DataFrame
 Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id, shop_name, item_category_name]
 Index: [],
 Empty DataFrame
 Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, item_name, item_category_id, shop_name, item_category_name]
 Index: [])